In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  
from librep.base.estimator import Estimator
%load_ext autoreload
%autoreload 2
from standartized_balanced import StandardizedBalancedDataset
import os,torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def get_data(dataset_name,sensors,normalize_data):    
    working_directory=f"result/{dataset_name}/"
    data_folder = f"../../../../../../experiment_executor/data/standartized_balanced/{dataset_name}/"
    dataset = StandardizedBalancedDataset(data_folder, sensors=sensors)
    X_train, y_train,X_test, y_test,X_val, y_val = dataset.get_all_data(normalize_data=normalize_data, resize_data=True)
    return X_train, y_train,X_test, y_test,X_val, y_val
        #print(f"shape: X_train {X_train.shape} --- X_test {X_test.shape} --- X_test {X_val.shape}")

In [2]:
X_train, y_train,X_test, y_test,X_val, y_val=get_data("MotionSense",['accel','gyro'],False)
input_shape=X_train[0].shape
print(input_shape)

(60, 6)


In [3]:
from librep.transforms.simclr import SimCLR


In [4]:
batch_size_head = 256
decay_steps_head = 1000
epochs_head = 300
temperature_head = 0.77
input_shape=X_train[0].shape
n_components=60
print(input_shape)
transform_funcs=['noise_vectorized', 'channel_shuffle_vectorized', 'amplify_attenuate']
verbose=1
import os,torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

simclr = SimCLR(input_shape=input_shape,n_components=n_components,batch_size=batch_size_head, transform_funcs=transform_funcs,
                                        temperature=temperature_head, epochs=epochs_head,
                                        is_transform_function_vectorized=True,
                                        verbose=verbose,device=device)

(60, 6)


In [5]:
trained_simclr_model,epoch_wise_loss = simclr.fit(X_train)


epoch: 1 loss: 5.750
epoch: 2 loss: 5.676
epoch: 3 loss: 5.665
epoch: 4 loss: 5.656
epoch: 5 loss: 5.627
epoch: 6 loss: 5.632
epoch: 7 loss: 5.442
epoch: 8 loss: 5.377
epoch: 9 loss: 5.363
epoch: 10 loss: 5.322
epoch: 11 loss: 5.320
epoch: 12 loss: 5.304
epoch: 13 loss: 5.291
epoch: 14 loss: 5.289
epoch: 15 loss: 5.307
epoch: 16 loss: 5.280
epoch: 17 loss: 5.283
epoch: 18 loss: 5.262
epoch: 19 loss: 5.263
epoch: 20 loss: 5.267
epoch: 21 loss: 5.264
epoch: 22 loss: 5.236
epoch: 23 loss: 5.237
epoch: 24 loss: 5.244
epoch: 25 loss: 5.223
epoch: 26 loss: 5.246
epoch: 27 loss: 5.208
epoch: 28 loss: 5.225
epoch: 29 loss: 5.201
epoch: 30 loss: 5.198
epoch: 31 loss: 5.192
epoch: 32 loss: 5.194
epoch: 33 loss: 5.202
epoch: 34 loss: 5.196
epoch: 35 loss: 5.195
epoch: 36 loss: 5.198
epoch: 37 loss: 5.197
epoch: 38 loss: 5.195
epoch: 39 loss: 5.187
epoch: 40 loss: 5.188
epoch: 41 loss: 5.177
epoch: 42 loss: 5.181
epoch: 43 loss: 5.175
epoch: 44 loss: 5.180
epoch: 45 loss: 5.187
epoch: 46 loss: 5.1

In [6]:
print(simclr.model)

SimCLRHead(
  (base_model): BaseModel(
    (conv1): Conv1d(6, 32, kernel_size=(24,), stride=(1,))
    (dropout1): Dropout(p=0.1, inplace=False)
    (conv2): Conv1d(32, 64, kernel_size=(16,), stride=(1,))
    (dropout2): Dropout(p=0.1, inplace=False)
    (conv3): Conv1d(64, 60, kernel_size=(8,), stride=(1,))
    (dropout3): Dropout(p=0.1, inplace=False)
    (global_max_pooling): AdaptiveMaxPool1d(output_size=1)
    (relu): ReLU()
  )
  (projection_1): Linear(in_features=60, out_features=256, bias=True)
  (relu): ReLU()
  (projection_2): Linear(in_features=256, out_features=128, bias=True)
  (projection_3): Linear(in_features=128, out_features=50, bias=True)
)


In [7]:
data=simclr.transform(X_train)
data_t=simclr.transform(X_test)
print(data.shape)
print(X_train.shape)
from sklearn.ensemble import RandomForestClassifier

# Crear un clasificador Random Forest
clf = RandomForestClassifier(n_estimators=200, random_state=42)  # Puedes ajustar los hiperparámetros según sea necesario

# Entrenar el clasificador
clf.fit(data, y_train)
etiquetas_pred = clf.predict(data_t)
from sklearn.metrics import accuracy_score, classification_report
accuracy = accuracy_score(y_test, etiquetas_pred)
report = classification_report(y_test, etiquetas_pred)
print(accuracy,report)

(3558, 60)
(3558, 60, 6)
0.8832391713747646               precision    recall  f1-score   support

           0       0.91      0.81      0.85       177
           1       0.83      0.91      0.87       177
           2       0.79      0.95      0.86       177
           3       0.91      0.76      0.83       177
           4       0.90      0.91      0.90       177
           5       0.99      0.96      0.97       177

    accuracy                           0.88      1062
   macro avg       0.89      0.88      0.88      1062
weighted avg       0.89      0.88      0.88      1062



In [8]:
data=simclr.predict(X_train)
data_t=simclr.predict(X_test)
print(data.shape)
print(X_train.shape)
from sklearn.ensemble import RandomForestClassifier

# Crear un clasificador Random Forest
clf = RandomForestClassifier(n_estimators=100, random_state=42)  # Puedes ajustar los hiperparámetros según sea necesario

# Entrenar el clasificador
clf.fit(data, y_train)
etiquetas_pred = clf.predict(data_t)
from sklearn.metrics import accuracy_score, classification_report
accuracy = accuracy_score(y_test, etiquetas_pred)
report = classification_report(y_test, etiquetas_pred)
print(accuracy,report)

(3558, 50)
(3558, 60, 6)
0.8559322033898306               precision    recall  f1-score   support

           0       0.82      0.77      0.80       177
           1       0.78      0.82      0.80       177
           2       0.81      0.92      0.86       177
           3       0.90      0.75      0.82       177
           4       0.84      0.92      0.87       177
           5       1.00      0.95      0.98       177

    accuracy                           0.86      1062
   macro avg       0.86      0.86      0.86      1062
weighted avg       0.86      0.86      0.86      1062

